In [1]:
""" !pip install -U pyspark
from pyspark.sql import SparkSession
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable """

" !pip install -U pyspark\nfrom pyspark.sql import SparkSession\nimport os\nimport sys\nos.environ['PYSPARK_PYTHON'] = sys.executable\nos.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable "

# StreamQuest Movie Plot Searching System
This application is created for movie studios to check the latest trends in the movie industry......

## Section 1. Word2vec Movie Recommender Model Training
The first part of this notebook is dedicated to data cleaning and trainning the word2vec model to create the following three tools for the studio writers and executives: 
1. Basic movie recommender: The user input one movie; and the system recommends 10 other movies with similar plotlines. 
2. Advance movie recommender: The user input two movies; and the system recommends 10 other movies with plotlines that are similar to the combination of these two movies. 
3. Duplicate plot checker: The user input his/her script for a new movie idea, and the system checks if his/her idea has already been produced in a previous movie. 


### 1.1 Data cleaning
There are three data sources used in this section: 
1. IMDB: used for matching movie name and ID
2. Details: contains plot summaries, synopsis and movie ID, both summaries and synopsis will be used for trainning
3. Wiki_Plot: contains plots and movie name, used for trainning

Note that there might be multiple entries for a same movie because multiple authors might have written summaries for it. 

In [2]:
path_to_imdb_dataset = '/Users/yupan/Library/CloudStorage/OneDrive-Personal/Academic/5430/data/title.basics.tsv.gz'
path_to_plots_dataset = '/Users/yupan/Library/CloudStorage/OneDrive-Personal/Academic/5430/data/wiki_movie_plots_deduped.csv'
path_to_details_dataset = '/Users/yupan/Library/CloudStorage/OneDrive-Personal/Academic/5430/data/IMDB_movie_details.json'

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()
print("Using Apache Spark Version", spark.version)

23/08/11 09:55:18 WARN Utils: Your hostname, Yus-MacBook-Air-2.local resolves to a loopback address: 127.0.0.1; using 192.168.181.65 instead (on interface en0)
23/08/11 09:55:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/11 09:55:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Using Apache Spark Version 3.4.1


In [4]:
# clean & combine the IMDB dataset with details dataset 
# reading the IMDB dataset
imdb = spark.read.options(header = True, inferSchema = True, delimiter = "\t")\
  .csv(path_to_imdb_dataset)
# filter the imdb dataset so that only movies are included
imdb = imdb.filter("titleType = 'movie'")\
  .select('tconst', 'primaryTitle', 'startYear')\
    .withColumnRenamed('startYear', 'Year')\
      .withColumnRenamed('primaryTitle', 'Title')\
        .dropDuplicates(['Title', 'Year'])
print('there is a total of ', imdb.count(), ' movies in the imdb dataset')


# reading the details dataset, preserving only three important variables
details_summary = spark.read.json(path_to_details_dataset)
details_summary = details_summary\
  .select('movie_id','plot_summary')\
    .withColumnRenamed('plot_summary','Plot')

# reading the details dataset, preserving only three important variables
details_synopsis = spark.read.json(path_to_details_dataset)
details_synopsis = details_synopsis.select('movie_id','plot_synopsis')\
  .filter("plot_synopsis != ''")\
    .withColumnRenamed('plot_synopsis', 'Plot')

details = details_summary.union(details_synopsis)
print('there is a total of ', details.count(), ' plot descriptions in the details dataset')


from pyspark.sql.functions import lit
# join the imdb with details by matching the unique identifier(e.g. tt0000000)
imdb_join_details = imdb.join(details, imdb.tconst == details.movie_id, 'inner')\
  .withColumnRenamed('tconst', 'id')\
    .select('id', 'Title', 'Plot')\
      .withColumn("Source", lit("imdb_details"))

print("The joined dataset has ", imdb_join_details.count(), " entries")
# inspect the joined dataset
imdb_join_details.show(3)

there is a total of  637758  movies in the imdb dataset
there is a total of  2911  plot descriptions in the details dataset


The joined dataset has  2857  entries


+---------+--------------+--------------------+------------+
|       id|         Title|                Plot|      Source|
+---------+--------------+--------------------+------------+
|tt2294449|22 Jump Street|Following their s...|imdb_details|
|tt2294449|22 Jump Street|After making thei...|imdb_details|
|tt0120623|  A Bug's Life|On a small island...|imdb_details|
+---------+--------------+--------------------+------------+
only showing top 3 rows



In [5]:
# clean and combine wiki plot dataset with imdb dataset
from pyspark.sql.functions import length
# reading the plot dataset, preserving only three important variables
wiki_plot = spark.read.options(header = True, inferSchema = True, quote = '"', escape = '"', multiLine = True).csv(path_to_plots_dataset)
wiki_plot = wiki_plot.select('Title', 'Release Year','Plot')\
  .withColumnRenamed('Release Year', 'Year')\
    .filter(length(wiki_plot['Plot']) >= 200) # filter out the very short plot descriptions
print('there is a total of ', wiki_plot.count(), ' plot descriptions in the wiki_plot dataset')


# join the imdb with the plot dataset by matching movie titles and release year
imdb_join_plot = imdb.join(wiki_plot, ["Title", "Year"], 'inner')\
  .withColumnRenamed('tconst', 'id')\
    .select('id', 'Title', 'Plot')\
      .withColumn("Source", lit("wiki_plot"))

print("The joined dataset has ", imdb_join_plot.count(), " entries")
# inspect the joined dataset
imdb_join_plot.show(1)

there is a total of  33243  plot descriptions in the wiki_plot dataset


The joined dataset has  25361  entries


+---------+-----+--------------------+---------+
|       id|Title|                Plot|   Source|
+---------+-----+--------------------+---------+
|tt0790799|$9.99|The film mainly f...|wiki_plot|
+---------+-----+--------------------+---------+
only showing top 1 row



In [6]:
# combine the above two dataset to get the dataset that we will train the model on
df = imdb_join_plot.union(imdb_join_details)

print('after merging & cleaning, there is a total of ', df.count(), ' movie plot entries left in the merged dataset')
# inspect the combined new dataset
df.show(1)

after merging & cleaning, there is a total of  28218  movie plot entries left in the merged dataset


+---------+-----+--------------------+---------+
|       id|Title|                Plot|   Source|
+---------+-----+--------------------+---------+
|tt0790799|$9.99|The film mainly f...|wiki_plot|
+---------+-----+--------------------+---------+
only showing top 1 row



### 1.2 Trainning Word2vec Model 
This Word2Vec model is trained using the cleaned data above, the model is fed with around 28,000 entries of texts that describe movie plots. The resulting model will be useful in finding similarities in movie plotlines. 

In [7]:
# tokenize and remove stop words in this cell
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec

# create a new field by copying Plot
df = df.withColumn('inputText', F.col('Plot')) 

# regular expression tokenizer to tokenize inputText into individual tokens (words)
regextok = RegexTokenizer(gaps = False, pattern = '\w+', inputCol = 'inputText', outputCol = 'tokens')

# StopWordsRemover to remove stopwords in the list of tokens
stopwrmv = StopWordsRemover(inputCol = 'tokens', outputCol = 'tokens_sw_removed')
df = regextok.transform(df)
df = stopwrmv.transform(df)
df.show(1)

+---------+-----+--------------------+---------+--------------------+--------------------+--------------------+
|       id|Title|                Plot|   Source|           inputText|              tokens|   tokens_sw_removed|
+---------+-----+--------------------+---------+--------------------+--------------------+--------------------+
|tt0790799|$9.99|The film mainly f...|wiki_plot|The film mainly f...|[the, film, mainl...|[film, mainly, fo...|
+---------+-----+--------------------+---------+--------------------+--------------------+--------------------+
only showing top 1 row



In [8]:
# train word2vec model, the parameters here can be changed to optimize the model
word2vec = Word2Vec(vectorSize = 100, minCount = 5, inputCol = 'tokens_sw_removed', outputCol = 'wordvectors')
model = word2vec.fit(df)

# using transform to add wordvectors column to dataframe
df = model.transform(df)
chunks = df.select('id', 'Title','wordvectors', 'Plot', 'Source').limit(30000).collect()

23/08/11 09:58:31 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [9]:
# define function to calculate cosine similarity for later
import numpy as np
def cossim(v1, v2): 
  '''
      cossim(v1, v2) calculates the cosine similarity between v1 and v1.
      If v1 or v2 is a zero vector, it will return 0
  '''
  if np.dot(v1, v1) == 0 or np.dot(v2, v2) == 0:
      return 0.0
  return float(np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))))

### 1.3 Create Basic, Advanced Recommender and Duplicate Plot Checker
These three tools use the same model at their core. Therefore, to optimize performance, the implementation will create only a single session when running. 

In [10]:
# writing a function to obtain the plot string from the plot dataset
def acquire_plot(base_movie: str): 
  # input: a movie name (precise) or a movie id 
  # output: the movie's plot

  if base_movie.startswith("tt"):   # search by movie name
    base_movie_row = df.filter(df.id == base_movie).collect()
  else:                             # search by movie id
    base_movie_row = df.filter(df.Title == base_movie).collect()

  if base_movie_row: 
    movie_plot = base_movie_row[0]['Plot']
    return movie_plot
  else: 
    print("Sorry, ", base_movie, " is not found in the database. Please type in exact movie names")

In [11]:
def query_preprocessing(plot: str): 
  plot_df = spark.createDataFrame([(1, plot)]).toDF('index','inputText')
  plot_tok = regextok.transform(plot_df)
  plot_swr = stopwrmv.transform(plot_tok)
  plot_vec = model.transform(plot_swr)
  plot_vec = plot_vec.select('wordvectors').collect()[0][0]
  return plot_vec

### 1.3.1 Basic recommender
The Basic Recommender only takes in one movie as the only parameter: 

In [12]:
input_user_1 = '2012'    # User input The exact movie name or movie id(e.g. 'tt1023003')
basic_movie_plot = acquire_plot(input_user_1)
basic_vec = query_preprocessing(basic_movie_plot)

### 1.3.2 Advanced Recommender
The Advanced Recommender takes in one extra movie as the second parameter, then our word2vec model will be able to recommend a third movie that has similar plot as the combination of the first two. : 

In [13]:
input_user_2 = '' #'tt0468569'    # User input The exact movie name or movie id(e.g. 'tt1023003')
if input_user_2: 
  second_movie_plot = acquire_plot(input_user_2)
  second_vec = query_preprocessing(second_movie_plot)
  combined_vec = basic_vec + second_vec

### 1.3.3 Duplicate Plot Checker
The duplicate Plot checker is similar to the basic recommender where it finds an existing movie with similar plot as the user's input plot description. 

In [14]:
input_user_3 = '' #'Once upon a time in a crime ridden Gotham City, a member of the rich Wayne family decided to put on a mask and protect the people of Gotham'
check_vec = query_preprocessing(input_user_3)

### 1.4 implementation of the above three tools
The parameters of the three tools have been created. To optimize performance, the implementation will create only one single session when running. 

__Note for Meenu__ : the order of the IF clauses in this following cell is important, the user can only run one of the three tools at once. 

In [15]:
output_lim = 10

if input_user_3: 
  print("Running Duplicate Plot Checker")
  data = [(i[0], float(cossim(check_vec, i[2])), i[1], i[4], i[3]) for i in chunks]
  sim_df = spark.createDataFrame(data).toDF('movie_id', 'similarity', 'Title', 'Source', 'Plot')
  sim_df = (sim_df.dropDuplicates(['movie_id'])
            .orderBy('similarity', ascending=False)
            .limit(output_lim))

elif not input_user_2:  # if input_user_2 is empty, then run the basic recommender
  
  print("Only one movie is input, running Basic Recommender")
  data = [(i[0], float(cossim(basic_vec, i[2])), i[1], i[4], i[3]) for i in chunks]
  sim_df = spark.createDataFrame(data).toDF('movie_id', 'similarity', 'Title', 'Source', 'Plot')
  sim_df = (sim_df.filter((sim_df.Title != input_user_1) & (sim_df.movie_id != input_user_1))
            .dropDuplicates(['movie_id'])
            .orderBy('similarity', ascending=False)
            .limit(output_lim))
  
elif input_user_2: 
  
  print("Two movies are input, running Advanced Recommender")
  data = [(i[0], float(cossim(combined_vec, i[2])), i[1], i[4], i[3]) for i in chunks]
  sim_df = spark.createDataFrame(data).toDF('movie_id', 'similarity', 'Title', 'Source', 'Plot')
  sim_df = (sim_df.filter((sim_df.Title != input_user_1) & (sim_df.movie_id != input_user_1)
                          & (sim_df.Title != input_user_2) & (sim_df.movie_id != input_user_2))
            .dropDuplicates(['movie_id'])
            .orderBy('similarity', ascending=False)
            .limit(output_lim))

Only one movie is input, running Basic Recommender


In [16]:
import torch
from transformers import BertTokenizer, BertModel
from torch.nn.functional import cosine_similarity

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    # Tokenize input, get output from BERT
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use mean pooling on the token embeddings to get sentence embeddings
    sentence_embedding = torch.mean(outputs.last_hidden_state, dim=1).squeeze(dim=0)
    return sentence_embedding

def calculate_similarity(text1, text2):
    embedding1 = get_bert_embedding(text1)
    embedding2 = get_bert_embedding(text2)
    similarity = cosine_similarity(embedding1.unsqueeze(0), embedding2.unsqueeze(0)).item()
    return similarity

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
accuracy_df = sim_df.toPandas()
def compute_scores(row):
    bert_score = calculate_similarity(row['Plot'], basic_movie_plot)
    bert_accuracy = (row['similarity'] - bert_score)*100 / bert_score
    return bert_score, bert_accuracy

accuracy_df[['BERT_score', 'BERT_delta%']] = accuracy_df.apply(lambda row: compute_scores(row), axis=1, result_type="expand")

In [29]:
accuracy_df

,movie_id,similarity,Title,Source,Plot,BERT_score,BERT_delta%
0,tt0319262,0.943857,The Day After Tomorrow,wiki_plot,"Jack Hall, an American paleoclimatologist, and...",0.963338,-2.022219
1,tt0377062,0.934648,Flight of the Phoenix,wiki_plot,When an Amacore oil rig in the Gobi Desert of ...,0.944732,-1.067381
2,tt0082810,0.929810,Night Crossing,wiki_plot,The film opens with a brief summary of 1961's ...,0.933364,-0.380793
3,tt0120647,0.927826,Deep Impact,wiki_plot,"On May 10, 1998, teenaged amateur astronomer L...",0.962801,-3.632629
4,tt4363250,0.927003,Lost in the Pacific,wiki_plot,"The year is 2020, two armed soldiers pass thro...",0.928369,-0.147097
5,tt0088172,0.923809,Starman,wiki_plot,"Launched in 1977, the Voyager 2 space probe ca...",0.935747,-1.275793
6,tt1470827,0.920738,Monsters,wiki_plot,After a NASA deep-space probe (sent to verify ...,0.945731,-2.642732
7,tt1860357,0.916612,Deepwater Horizon,wiki_plot,"On April 20, 2010, Deepwater Horizon, an oil d...",0.922323,-0.619277
8,tt0053137,0.915722,On the Beach,wiki_plot,"In early 1964, in the months following World W...",0.948907,-3.497158
9,tt0119567,0.915631,The Lost World: Jurassic Park,wiki_plot,"On Isla Sorna, an island off the Costa Rican c...",0.946503,-3.261696


In [30]:
accuracy_df['BERT_delta%'].mean()

-1.8546775057540987

In [ ]:
""" from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col, lit

@udf(FloatType())
def calculate_similarity_udf(text1, text2):
    return calculate_similarity(text1, text2)

sim_df = sim_df.withColumn("BERT Score", calculate_similarity_udf(col('Plot'), lit(basic_movie_plot)))
sim_df.show(output_lim, truncate=False) """